In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint
import setGPU
import gc
import math

setGPU: Setting GPU to: 1


In [2]:
#Definizione Modello
def getUnit1(layerPrec, filters, kernel_size = (1, 1), activation='relu', kernel_initializer='he_normal'):
  l = BatchNormalization()(layerPrec)
  l = Activation('relu')(l)
  l = Conv2D(filters, kernel_size, kernel_initializer=kernel_initializer, padding='same',use_bias=False)(l)
  return l

def getUnit2(layerPrec, filters, kernel_size = (3, 3), activation='relu', kernel_initializer='he_normal'):
  l = BatchNormalization()(layerPrec)
  l = Activation('relu')(l)
  l = Conv2D(filters, kernel_size, kernel_initializer=kernel_initializer, padding='same',use_bias=False)(l)
  return l

def getDownBlock(layerPrec,m,n,indexBlock):
  l = getUnit1(layerPrec,4*n)
  l = getUnit2(l,n)
  nets["layers"][f"down{indexBlock}"].append(l)

  concat = Concatenate()([layerPrec,l])
  l = getUnit1(concat,m)
  maxPooling = MaxPool2D(padding='same')(l)
  return maxPooling, getUnit1(concat,m)

def getUpBlock(layerPrec,skipConn,m,n,indexBlock,upLayers=[]):
  l = getUnit1(layerPrec,m)
  l = UpSampling2D()(layerPrec)
  concat = Concatenate()([skipConn,l]+upLayers)
  l = getUnit1(concat,4*n)
  l = getUnit2(l,n)
  nets["layers"][f"up{indexBlock}"].append(l)
  concat = Concatenate()([concat,l])
  return concat

def getUNet(input,m,n,indexUNet,nUNet, nBlocks):
  layerPrec = input
  listSkipConn = []

  if indexUNet != 0:
    layerPrec = nets[f"unet{indexUNet-1}"]
    #layerPrec = Concatenate()([input,layerPrec]) #l'abbiamo fatto giù con l'if dopo l'up
    layerPrec = getUnit1(layerPrec,m)

  # down
  for i in range(nBlocks):
    if nets["layers"][f"down{i}"]:
        layerPrec = Concatenate()([layerPrec]+nets["layers"][f"down{i}"])
    layerPrec, skipConn = getDownBlock(layerPrec,m,n,i)
    listSkipConn.append(skipConn)

  # bottle neck
  if nets["layers"][f"bn"]:
      layerPrec = Concatenate()([layerPrec]+nets["layers"][f"bn"])

  l = getUnit1(layerPrec,4*n)
  l = getUnit2(l,n)
  nets["layers"]["bn"].append(l)
  concat = Concatenate()([layerPrec,l])

  # up
  layerPrec = concat
  for i in range(nBlocks):
    layerPrec = getUpBlock(layerPrec,listSkipConn[-(i+1)],m,n,i,upLayers=nets["layers"][f"up{i}"])


  if indexUNet != nUNet - 1:
    l = Concatenate()([input,layerPrec])
  else:
    l = getUnit1(layerPrec,16,activation="sigmoid") #era linear

  return l

def trasformationInput(x, filters):
  #x = BatchNormalization()(x)
  #x = Activation('relu')(x)
  #x = Conv2D(filters, kernel_size=(7,7), strides=(2,2), kernel_initializer="he_normal", padding='same',use_bias=False)(x)
  #maxPooling = MaxPool2D(padding='same')(x)
  
  x = Conv2D(filters*2, kernel_size=(5,5), strides=(1,1), kernel_initializer="he_normal", padding='same',use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(filters, kernel_size=(3,3), strides=(1,1), kernel_initializer="he_normal", padding='same',use_bias=False)(x) #era 3x3
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPool2D(padding='same')(x)

  return x#maxPooling

def getCUNet(shape,m,n,nUNet,nBlocks):
  for i in range(nUNet):
    nets[f"unet{i}"] = None

  for j in range(nBlocks):
    nets["layers"][f"down{j}"] = []
    nets["layers"][f"up{j}"] = []
    
  input = Input(shape=shape)
  
  t_input = trasformationInput(input,m) # per le heatmap da 64x64

  for i in range(nUNet):
    nets[f"unet{i}"] = getUNet(t_input,m,n,i,nUNet,nBlocks)

  output = nets[f"unet{nUNet-1}"]
  return Model(inputs=input, outputs=output)

nets = {}
nets["layers"] = {}
nets["layers"]["bn"] = []
shape = (128,128,3)
m = 128
n = 24
nUNet = 2
nBlocks = 4
#mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
#with mirrored_strategy.scope():
net = getCUNet(shape,m,n,nUNet,nBlocks)

layers = net.layers
#print(layers[57].input)
for i in range(len(layers)):
    if isinstance(layers[i], tf.python.keras.layers.convolutional.Conv2D):
        if isinstance(layers[i-1].input,list):
            in_chan = 0
            for k in layers[i-1].input:
                in_chan = in_chan + k.shape[-1]
            print(in_chan)
        else:
            in_chan = layers[i-1].input.shape[-1]
            
        n1 = layers[i].kernel_size[0] * layers[i].kernel_size[1] * in_chan
        stdv = 1/math.sqrt(n1)
        layers[i].kernel_initializer = tf.keras.initializers.RandomUniform(minval=-stdv, maxval=stdv)

net.summary()

152
304
456
608
176
352
528
704
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 256 19200       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 128, 128, 256 1024        conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 128, 128, 256 0           batch_normalization[0][0]        
_______________________________________________________

In [3]:
def get_max_preds(heatmaps):
    """Get keypoint predictions from score maps.
    Note:
        batch_size: N
        num_keypoints: K
        heatmap height: H
        heatmap width: W
    Args:
        heatmaps (np.ndarray[N, K, H, W]): model predicted heatmaps.
    Returns:
        tuple: A tuple containing aggregated results.
        - preds (np.ndarray[N, K, 2]): Predicted keypoint location.  
        - maxvals (np.ndarray[N, K, 1]): Scores (confidence) of the keypoints. non lo restituiamo
    """
    #assert isinstance(heatmaps, np.ndarray), ('heatmaps should be numpy.ndarray')
    #assert heatmaps.ndim == 4, 'batch_images should be 4-ndim'

    N, K, _, W = heatmaps.shape
    heatmaps_reshaped = heatmaps.reshape((N, K, -1))
    idx = np.argmax(heatmaps_reshaped, 2).reshape((N, K, 1))
    maxvals = np.amax(heatmaps_reshaped, 2).reshape((N, K, 1))

    preds = np.tile(idx, (1, 1, 2)).astype(np.float32)
    preds[:, :, 0] = preds[:, :, 0] % W
    preds[:, :, 1] = preds[:, :, 1] // W

    preds = np.where(np.tile(maxvals, (1, 1, 2)) > 0.0, preds, -1)
    return preds#, maxvals

def calc_dists(preds, target, normalize, use_zero=False):
    preds = preds.astype(np.float32)
    target = target.astype(np.float32)
    normalize = normalize.astype(np.float32)
    dists = np.zeros((preds.shape[1], preds.shape[0]))
    if use_zero:
        boundary = 0
    else:
        boundary = 1
    for n in range(preds.shape[0]):
        for c in range(preds.shape[1]):
            if target[n,c,0] > boundary and target[n, c, 1] > boundary:
                dists[c, n] = np.linalg.norm((preds[n,c,:]- target[n,c,:])/normalize[n]) # axis ricavato da solo
            else:
                dists[c, n] = -1
    return dists

def dist_acc(dists, thr=0.5):

  #Return percentage below threshold while ignoring values with a -1

  if (dists!=-1).sum() > 0:

    return ((dists<=thr) == (dists!=-1)).sum().astype(np.float32) / (dists!=-1).sum().astype(np.float32)

  else:

    return -1

def accuracy(output, target, thr=0.5):
  ''' Calculate accuracy according to PCK, but uses ground truth heatmap rather than x,y locations
        First value to be returned is average accuracy across 'idxs', followed by individual accuracies
    '''
  #output = output.numpy()
  #print(output.__class__)
  #target = target.numpy()

  idkp = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
  preds = get_max_preds(output)
  gts = get_max_preds(target)
  norm = np.ones(preds.shape[0])*output.shape[3]/10

  dists = calc_dists(preds, gts, norm)

  acc = np.zeros(len(idkp)+1)
  avg_acc = 0
  cnt = 0

  for i in range(len(idkp)):
    acc[i+1] = dist_acc(dists[idkp[i]])
    if acc[i+1] >= 0: 
      avg_acc = avg_acc + acc[i+1]
      cnt += 1
            
  if cnt != 0:  
    acc[0] = avg_acc / cnt

  return acc

In [4]:
import numpy as np
import scipy.misc
import cv2

# =============================================================================
# General image processing functions
# =============================================================================

def get_transform(center, scale, res, rot=0):
    # Generate transformation matrix
    h = 200 * scale
    t = np.zeros((3, 3))
    t[0, 0] = float(res[1]) / h
    t[1, 1] = float(res[0]) / h
    t[0, 2] = res[1] * (-float(center[0]) / h + .5)
    t[1, 2] = res[0] * (-float(center[1]) / h + .5)
    t[2, 2] = 1
    if not rot == 0:
        rot = -rot # To match direction of rotation from cropping
        rot_mat = np.zeros((3,3))
        rot_rad = rot * np.pi / 180
        sn,cs = np.sin(rot_rad), np.cos(rot_rad)
        rot_mat[0,:2] = [cs, -sn]
        rot_mat[1,:2] = [sn, cs]
        rot_mat[2,2] = 1
        # Need to rotate around center
        t_mat = np.eye(3)
        t_mat[0,2] = -res[1]/2
        t_mat[1,2] = -res[0]/2
        t_inv = t_mat.copy()
        t_inv[:2,2] *= -1
        t = np.dot(t_inv,np.dot(rot_mat,np.dot(t_mat,t)))
    return t

def transform(pt, center, scale, res, invert=0, rot=0):
    # Transform pixel location to different reference
    t = get_transform(center, scale, res, rot=rot)
    if invert:
        t = np.linalg.inv(t)
    new_pt = np.array([pt[0], pt[1], 1.]).T
    new_pt = np.dot(t, new_pt)
    return new_pt[:2].astype(int)

def crop(img, center, scale, res, rot=0):
    # Upper left point
    ul = np.array(transform([0, 0], center, scale, res, invert=1))
    # Bottom right point
    br = np.array(transform(res, center, scale, res, invert=1))

    new_shape = [br[1] - ul[1], br[0] - ul[0]]
    if len(img.shape) > 2:
        new_shape += [img.shape[2]]
    new_img = np.zeros(new_shape)

    # Range to fill new array
    new_x = max(0, -ul[0]), min(br[0], len(img[0])) - ul[0]
    new_y = max(0, -ul[1]), min(br[1], len(img)) - ul[1]
    # Range to sample from original image
    old_x = max(0, ul[0]), min(len(img[0]), br[0])
    old_y = max(0, ul[1]), min(len(img), br[1])
    new_img[new_y[0]:new_y[1], new_x[0]:new_x[1]] = img[old_y[0]:old_y[1], old_x[0]:old_x[1]]

    return cv2.resize(new_img, res)

def inv_mat(mat):
    ans = np.linalg.pinv(np.array(mat).tolist() + [[0,0,1]])
    return ans[:2]

def kpt_affine(kpt, mat):
    kpt = np.array(kpt)
    shape = kpt.shape
    kpt = kpt.reshape(-1, 2)
    return np.dot( np.concatenate((kpt, kpt[:, 0:1]*0+1), axis = 1), mat.T ).reshape(shape)


def resize(im, res):
    return np.array([cv2.resize(im[i],res) for i in range(im.shape[0])])

def generateHeatmap(keypoints,output_res,num_parts):
    #Init
    sigma = output_res/64
    size = 6*sigma+3
    x = np.arange(0, size, 1, float) #crea un array composto da nove 1.0 vettore 1-D (9,)
    y = x[:, np.newaxis] #Vettore 2-D (9,1)
    x0, y0 = 3*sigma + 1, 3*sigma + 1 #vedi giù a tutto
    g = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
    #Generation
    hms = np.zeros(shape = (num_parts, output_res, output_res), dtype = np.float32) #crea vettore (16,64,64), cioè 16 heatmaps nere
    for p in keypoints:
        for idx, pt in enumerate(p): #ottiene id + [x,y] di ogni keypoint
            if pt[0] > 0: 
                x, y = int(pt[0]), int(pt[1])
                if x<0 or y<0 or x>=output_res or y>=output_res: #se succede questo, rimane heatmap idx-esima  tutta a 0
                    continue
                ul = int(x - 3*sigma - 1), int(y - 3*sigma - 1)
                br = int(x + 3*sigma + 2), int(y + 3*sigma + 2)

                c,d = max(0, -ul[0]), min(br[0], output_res) - ul[0]
                a,b = max(0, -ul[1]), min(br[1], output_res) - ul[1]

                cc,dd = max(0, ul[0]), min(br[0], output_res)
                aa,bb = max(0, ul[1]), min(br[1], output_res)
                hms[idx, aa:bb,cc:dd] = np.maximum(hms[idx, aa:bb,cc:dd], g[a:b,c:d])
    return hms
def getImgHms(img,c,s,keypoints,inp_res=(256,256),out_res=(64,64)):
    cropped = crop(img,c,s,inp_res)
    orig_keypoints = []
    for i in keypoints:
        orig_keypoints.append(np.array([i["x"],i["y"]]))
    orig_keypoints = np.array(orig_keypoints).reshape((1,16,2))
    kptmp = np.copy(orig_keypoints)
    for i in range(orig_keypoints.shape[1]):
        if orig_keypoints[0,i,0] > 0:
            orig_keypoints[0,i,:2] = transform(orig_keypoints[0,i,:2], c, s, inp_res)
    keypoints = np.copy(orig_keypoints)
    h, w = cropped.shape[0:2]
    center = np.array((w/2,h/2))
    scale = max(h,w)/200
    aug_rot = (np.random.random()*2-1)*30
    aug_scale = np.random.random() * (1.25-0.75)+0.75
    scale *= aug_scale

    mat_mask = get_transform(center, scale, out_res, aug_rot)[:2]
    mat = get_transform(center, scale, inp_res, aug_rot)[:2]
    inp = cv2.warpAffine(cropped, mat, inp_res).astype(np.float32)/255

    keypoints[:,:,0:2] = kpt_affine(keypoints[:,:,0:2], mat_mask) 
    
    #print('\n')


    #Flip 50% probability
    if np.random.randint(2) == 0:
        inp = inp[:, ::-1]
        keypoints = keypoints[:, flipped_parts['mpii']]
        keypoints[:, :, 0] = 64 - keypoints[:, :, 0]
        orig_keypoints = orig_keypoints[:, flipped_parts['mpii']]
        orig_keypoints[:, :, 0] = 256 - orig_keypoints[:, :, 0]

    ## set keypoints to 0 when were not visible initially (so heatmap all 0s)
        for i in range(np.shape(orig_keypoints)[1]):
            if kptmp[0,i,0] == 0 and kptmp[0,i,1] == 0:
                keypoints[0,i,0] = 0
                keypoints[0,i,1] = 0
                orig_keypoints[0,i,0] = 0
                orig_keypoints[0,i,1] = 0

    heatmaps = generateHeatmap(keypoints,out_res[0],16)
    return inp,heatmaps

In [5]:
#optimizer = tf.keras.optimizers.RMSprop(
#    learning_rate=6.7e-3, rho=0.99, momentum=0.0, epsilon=1e-08
#)
optimizer = tf.keras.optimizers.Adam(learning_rate=6.7e-3)


def heatmapLoss(y_true,y_pred):
    l = tf.math.square((y_pred - y_true))
    l = tf.reduce_mean(l,axis=3)
    l = tf.reduce_mean(l,axis=2)
    l = tf.reduce_mean(l,axis=1)
    return l

def euclidean_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_true - y_pred)))

def dice(y_true,y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

def dice_loss(y_true, y_pred):
    return 1 - dice(y_true, y_pred)

def weighted_mse_loss(y_true, y_pred):
    """
    apply weights on heatmap mse loss to only pick valid keypoint heatmap
    since y_true would be gt_heatmap with shape
    (batch_size, heatmap_size[0], heatmap_size[1], num_keypoints)
    we sum up the heatmap for each keypoints and check. Sum for invalid
    keypoint would be 0, so we can get a keypoint weights tensor with shape
    (batch_size, 1, 1, num_keypoints)
    and multiply to loss
    """
    heatmap_sum = K.sum(K.sum(y_true, axis=1, keepdims=True), axis=2, keepdims=True)

    # keypoint_weights shape: (batch_size, 1, 1, num_keypoints), with
    # valid_keypoint = 1.0, invalid_keypoint = 0.0
    keypoint_weights = 1.0 - K.cast(K.equal(heatmap_sum, 0.0), 'float32')

    return K.sqrt(K.mean(K.square((y_true - y_pred) * keypoint_weights)))

def smooth_l1_loss(y_true, y_pred):
    diff = K.abs(y_true - y_pred)
    less_than_one = K.cast(K.less(diff, 1.0), 'float32')
    loss = (less_than_one * 0.5 * diff**2) + (1 - less_than_one) * (diff - 0.5)

    return loss


parts = {'mpii':['rank', 'rkne', 'rhip',
                 'lhip', 'lkne', 'lank',
                 'pelv', 'thrx', 'neck', 'head',
                 'rwri', 'relb', 'rsho',
                 'lsho', 'lelb', 'lwri']}

flipped_parts = {'mpii':[5, 4, 3, 2, 1, 0, 6, 7, 8, 9, 15, 14, 13, 12, 11, 10]}

part_pairs = {'mpii':[[0, 5], [1, 4], [2, 3], [6], [7], [8], [9], [10, 15], [11, 14], [12, 13]]}

pair_names = {'mpii':['ankle', 'knee', 'hip', 'pelvis', 'thorax', 'neck', 'head', 'wrist', 'elbow', 'shoulder']}

In [6]:
import json
import imageio

class Dataset(tf.keras.utils.Sequence):
    #Costruttore
    def __init__(self,json_path,batch_size,input_shape=(256,256,3),output_shape=(64,64,16),train=True,dataset_name="MPII"):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.batch_size = batch_size
        f = open(json_path)
        self.dataset = json.load(f)
        if dataset_name == "MPII":
            self.dataset = self.dataset["dataset"]["MPII"]["people"]
        if train:
            self.dataset = self.dataset[:-7221]
        self.batch_images = np.zeros(shape=((self.batch_size,)+(self.input_shape)), dtype=np.float32)
        self.batch_heatmaps = np.zeros(shape=((self.batch_size,)+(self.output_shape)), dtype=np.float32)
        
    def __len__(self):
        return len(self.dataset) // self.batch_size
    
    def __getitem__(self,i):
        for n, annotation in enumerate(self.dataset[i*self.batch_size:(i+1)*self.batch_size]):
            #sample_index = i*self.batch_size + n
            image, gt_heatmap = self.extract_imgs_hms(annotation)
            gt_heatmap = np.transpose(gt_heatmap,(1,2,0))
            self.batch_images[n,:,:,:] = image
            self.batch_heatmaps[n, :, :, :] = gt_heatmap#gt_heatmap.reshape((self.output_shape))
            
        return self.batch_images, self.batch_heatmaps
#getImgHms(img,c,s,keypoints,inp_res=(256,256),out_res=(64,64)):
    
    def extract_imgs_hms(self,annotation):
        inp_res = self.input_shape[0:2]
        out_res = self.output_shape[0:2]
        img = imageio.imread(f"../images_mpii/{annotation['filepath']}")
        c = [annotation["objpos"]["x"],annotation["objpos"]["y"]]
        s = annotation["scale"]
        keypoints = annotation["keypoints"]
        cropped = crop(img,c,s,inp_res)
        
        orig_keypoints = []
        for i in keypoints:
            orig_keypoints.append(np.array([i["x"],i["y"]]))
        orig_keypoints = np.array(orig_keypoints).reshape((1,16,2))
        kptmp = np.copy(orig_keypoints)
        for i in range(orig_keypoints.shape[1]):
            if orig_keypoints[0,i,0] > 0:
                orig_keypoints[0,i,:2] = transform(orig_keypoints[0,i,:2], c, s, inp_res)
        keypoints = np.copy(orig_keypoints)
        h, w = cropped.shape[0:2]
        center = np.array((w/2,h/2))
        scale = max(h,w)/200
        aug_rot = (np.random.random()*2-1)*30
        aug_scale = np.random.random() * (1.25-0.75)+0.75
        scale *= aug_scale

        mat_mask = get_transform(center, scale, out_res, aug_rot)[:2]
        mat = get_transform(center, scale, inp_res, aug_rot)[:2]
        inp = cv2.warpAffine(cropped, mat, inp_res).astype(np.float32)/255

        keypoints[:,:,0:2] = kpt_affine(keypoints[:,:,0:2], mat_mask) 

        #print('\n')


        #Flip 50% probability
        if np.random.randint(2) == 0:
            inp = inp[:, ::-1]
            keypoints = keypoints[:, flipped_parts['mpii']]
            keypoints[:, :, 0] = 64 - keypoints[:, :, 0]
            orig_keypoints = orig_keypoints[:, flipped_parts['mpii']]
            orig_keypoints[:, :, 0] = 256 - orig_keypoints[:, :, 0]

        ## set keypoints to 0 when were not visible initially (so heatmap all 0s)
            for i in range(np.shape(orig_keypoints)[1]):
                if kptmp[0,i,0] == 0 and kptmp[0,i,1] == 0:
                    keypoints[0,i,0] = 0
                    keypoints[0,i,1] = 0
                    orig_keypoints[0,i,0] = 0
                    orig_keypoints[0,i,1] = 0

        heatmaps = generateHeatmap(keypoints,out_res[0],16)
        return inp,heatmaps

In [7]:
import json
import imageio

class Dataset2(tf.keras.utils.Sequence):
    #Costruttore
    def __init__(self,json_path,batch_size,input_shape=(256,256,3),output_shape=(64,64,16),train=True,dataset_name="MPII"):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.batch_size = batch_size
        self.images = np.load("../imgs_mpii.npz")['arr_0']
        self.hms = np.load("../hms_mpii.npz")['arr_0']
        self.hms = self.hms.reshape((self.hms.shape[0],64,64,16))
        if dataset_name == "MPII":
            self.images = self.images["dataset"]["MPII"]["people"]
        if train:
            self.images = self.images[:-8665]
        
    def __len__(self):
        return len(self.dataset) // self.batch_size
    
    def __getitem__(self,i):
        return self.images[i*self.batch_size:(i+1)*self.batch_size], self.hms[i*self.batch_size:(i+1)*self.batch_size]
#getImgHms(img,c,s,keypoints,inp_res=(256,256),out_res=(64,64)):
    
    def extract_imgs_hms(self,annotation):
        inp_res = self.input_shape[0:2]
        out_res = self.output_shape[0:2]
        img = imageio.imread(f"../images_mpii/{annotation['filepath']}")
        c = [annotation["objpos"]["x"],annotation["objpos"]["y"]]
        s = annotation["scale"]
        keypoints = annotation["keypoints"]
        cropped = crop(img,c,s,inp_res)
        
        orig_keypoints = []
        for i in keypoints:
            orig_keypoints.append(np.array([i["x"],i["y"]]))
        orig_keypoints = np.array(orig_keypoints).reshape((1,16,2))
        kptmp = np.copy(orig_keypoints)
        for i in range(orig_keypoints.shape[1]):
            if orig_keypoints[0,i,0] > 0:
                orig_keypoints[0,i,:2] = transform(orig_keypoints[0,i,:2], c, s, inp_res)
        keypoints = np.copy(orig_keypoints)
        h, w = cropped.shape[0:2]
        center = np.array((w/2,h/2))
        scale = max(h,w)/200
        aug_rot = (np.random.random()*2-1)*30
        aug_scale = np.random.random() * (1.25-0.75)+0.75
        scale *= aug_scale

        mat_mask = get_transform(center, scale, out_res, aug_rot)[:2]
        mat = get_transform(center, scale, inp_res, aug_rot)[:2]
        inp = cv2.warpAffine(cropped, mat, inp_res).astype(np.float32)/255

        keypoints[:,:,0:2] = kpt_affine(keypoints[:,:,0:2], mat_mask) 

        #print('\n')


        #Flip 50% probability
        if np.random.randint(2) == 0:
            inp = inp[:, ::-1]
            keypoints = keypoints[:, flipped_parts['mpii']]
            keypoints[:, :, 0] = 64 - keypoints[:, :, 0]
            orig_keypoints = orig_keypoints[:, flipped_parts['mpii']]
            orig_keypoints[:, :, 0] = 256 - orig_keypoints[:, :, 0]

        ## set keypoints to 0 when were not visible initially (so heatmap all 0s)
            for i in range(np.shape(orig_keypoints)[1]):
                if kptmp[0,i,0] == 0 and kptmp[0,i,1] == 0:
                    keypoints[0,i,0] = 0
                    keypoints[0,i,1] = 0
                    orig_keypoints[0,i,0] = 0
                    orig_keypoints[0,i,1] = 0

        heatmaps = generateHeatmap(keypoints,out_res[0],16)
        return inp,heatmaps
####################################################################àà


def get_normalize(input_shape):
    """
    rescale keypoint distance normalize coefficient
    based on input shape, used for PCK evaluation
    NOTE: 6.4 is standard normalize coefficient under
          input shape (256,256)
    # Arguments
        input_shape: input image shape as (height, width)
    # Returns
        scale: normalize coefficient
    """
    #assert input_shape[0] == input_shape[1], 'only support square input shape.'

    # use averaged scale factor for non square input shape
    scale = float((input_shape[0] + input_shape[1]) / 2) / 256.0

    return 6.4*scale

global_best_acc = 0.0

class EvalCallBack(tf.keras.callbacks.Callback):
    def __init__(self, model_input_shape, fileName, imgs,hms):
        self.normalize = get_normalize(model_input_shape)
        self.model_input_shape = model_input_shape
        self.best_acc = 0.0
        self.fileName = fileName
        self.eval_images = imgs#np.load("../imgs_val_mpii128.npz")['arr_0']
        self.eval_hms = hms#np.load("../hms_val_mpii128.npz")['arr_0']
        self.listAcc = []
        #self.eval_hms = np.reshape(self.eval_hms,(self.eval_hms.shape[0],64,64,16))

    def on_epoch_end(self, epoch, logs=None):
        global global_best_acc
        output = self.model.predict(self.eval_images)
        #output = output.reshape( (output.shape[0],)+(16,64,64) )
        output = np.transpose(output,(0,3,1,2))
        val_acc = accuracy(output,self.eval_hms)
        self.listAcc.append(val_acc[0])
        print('\nvalidate accuracy:\n', val_acc, '@epoch', epoch)
        f = open(self.fileName, "a")
        if val_acc[0] > self.best_acc:
            # Save best accuray value and model checkpoint
            #checkpoint_dir = os.path.join(self.log_dir, 'ep{epoch:03d}-loss{loss:.3f}-val_acc{val_acc:.3f}.h5'.format(epoch=(epoch+1), loss=logs.get('loss'), val_acc=val_acc))
            #self.model.save(f"../modelsave/ep{epoch}_acc{val_acc[0]}.h5")
            self.model.save_weights(f"../cunet{nUNet}_{m}{n}_best_sigmoidAdamQuasiDef3sx12_restartsig8_12824.h5")
            print('Epoch {epoch:03d}: val_acc improved from {best_acc:.3f} to {val_acc:.3f}'.format(epoch=epoch+1, best_acc=self.best_acc, val_acc=val_acc[0]))# checkpoint_dir=checkpoint_dir))
            message = 'Epoch {epoch:03d}: val_acc improved from {best_acc:.3f} to {val_acc:.3f}. Train loss is {loss:.3f}\n'.format(epoch=epoch+1, best_acc=self.best_acc, val_acc=val_acc[0],loss=logs.get('loss'))
            self.best_acc = val_acc[0]
            global_best_acc = val_acc[0]
            
            
        else:
            message = 'Epoch {epoch:03d}: val_acc did not improve from {best_acc:.3f}. Train loss is {loss:.3f}\n'.format(epoch=epoch+1, best_acc=self.best_acc,loss=logs.get('loss'))
            print('Epoch {epoch:03d}: val_acc did not improve from {best_acc:.3f}'.format(epoch=epoch+1, best_acc=self.best_acc))
        f.write(message)
        f.close()
        if epoch == 150:
            np.savez_compressed(f'../cunet{nUNet}_{m}{n}_best_sigmoidAdamQuasiDef3sx12_restartsig8_12824_accs', self.listAcc)
        gc.collect()
        
        
class CustomLearningRateScheduler(tf.keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self, patience,factor,fileName=None):
        super(CustomLearningRateScheduler, self).__init__()
        self.counter = 0
        self.patience = patience
        self.fileName = fileName
        self.factor = factor
        self.best_acc = 0.0

    def on_epoch_begin(self, epoch, logs=None):
        global global_best_acc
        print(f"Counter: {self.counter}, Global: {global_best_acc}, MyBest: {self.best_acc}\n")
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
    
    def schedule(self,epoch,lr):
        global global_best_acc
        if self.counter == self.patience:
            self.counter = 0 
            print("Epoch %03d: Updating Learning rate.. New value is %f" % (epoch, lr*self.factor))
            message = "Epoch %03d: Updating Learning rate.. New value is %f" % (epoch, lr*self.factor)
            f = open(self.fileName, "a")
            f.write(message)
            f.close()
            return lr*self.factor
        if self.best_acc == global_best_acc:
            self.counter = self.counter + 1
        elif self.best_acc < global_best_acc:
            self.counter = 0
            self.best_acc = global_best_acc
        return lr
    


In [8]:
import pickle 
with open('../imgs_train_mpii128sx12_genhm.pickle', 'rb') as handle:
    train_images = pickle.load(handle)
with open('../hms_train_mpii128sx12_genhm.pickle', 'rb') as handle:
    train_hms = pickle.load(handle)

eval_images = np.load("../imgs_val_mpii128sx12_genhm.npz")['arr_0']
eval_hms = np.load("../hms_val_mpii128sx12_genhm.npz")['arr_0']

#train_images = np.load("../imgs_train_mpii128.npz")['arr_0']
#train_hms = np.load("../hms_train_mpii128.npz")['arr_0']
train_hms = np.transpose(train_hms,(0,2,3,1))#np.reshape(train_hms,(train_hms.shape[0],64,64,16))
eval_hms2 = np.transpose(eval_hms,(0,2,3,1))
print("Finito 1")
"""
dataset = Dataset("datasets.json",32)

"""

Finito 1


'\ndataset = Dataset("datasets.json",32)\n\n'

In [9]:
evalcallback = EvalCallBack((128,128),"../sigmoidLayersAdamSchedulingQuasiDef3sx12_restartsig8_12824.txt",eval_images,eval_hms)
updateLR = CustomLearningRateScheduler(7,0.2,fileName="../sigmoidLayersAdamSchedulingQuasiDef3sx12_restartsig8_12824.txt")
print("Finito 2")


Finito 2


In [ ]:

net.compile(optimizer=optimizer, loss=weighted_mse_loss, metrics=[])
#checkpointer = ModelCheckpoint(filepath=f'../cunet{nUNet}_{m}{n}_best.h5', verbose=1, monitor='val_loss', mode='auto', save_best_only=True)
history = net.fit(train_images,train_hms,validation_data=(eval_images,eval_hms2),epochs=151, batch_size=32,shuffle=True,verbose=1, callbacks=[evalcallback,updateLR])#,use_multiprocessing=True,workers=20)#[checkpointer,updateLR])
#np.savez_compressed("../history",history)
net.save_weights(f'../cunet{nUNet}_{m}{n}_last_sigmoidAdamQuasiDef3sx12_restartsig8_12824.h5', overwrite=True)
np.save('../history_sigmoidAdamQuasiDef3sx12_restartsig8_12824.npy',history.history)

Counter: 0, Global: 0.0, MyBest: 0.0

Epoch 1/151
576/576 [==============================] - ETA: 0s - loss: 0.0998WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0116s vs `on_test_batch_end` time: 0.1015s). Check your callbacks.

validate accuracy:
 [0.13908344 0.00232706 0.09540576 0.22613719 0.20213534 0.07263882
 0.07346421 0.19907407 0.20164782 0.17681564 0.16529404 0.11338894
 0.14606741 0.14657936 0.16404182 0.14028929 0.10002825] @epoch 0
Epoch 001: val_acc improved from 0.000 to 0.139
576/576 [==============================] - 282s 490ms/step - loss: 0.0998 - val_loss: 0.1067
Counter: 1, Global: 0.1390834391786484, MyBest: 0.0

Epoch 2/151
576/576 [==============================] - ETA: 0s - loss: 0.0595
validate accuracy:
 [0.24687527 0.11381426 0.14520456 0.27537906 0.27441928 0.15853044
 0.10597424 0.30801505 0.46264488 0.50432962 0.3420026  0.18068711
 0.19929776 0.25860387 0.26703832 0.18466507 0.16939813] @epoch 1

In [ ]:
"""
net.load_weights("../cunet2_6416_last_sigmoidAdamQuasiDef3sx12.h5")
output = net.predict(train_images)
output = np.transpose(output,(0,3,1,2))
print(accuracy(output,np.transpose(train_hms,(0,3,1,2))))
"""

In [ ]:
#Ho aumentato solo patience 
#Provato con dropout ma andato male